In [1]:
# #ResNet 50
# from keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# import numpy as np

# model = ResNet50(weights='imagenet')

# img_path = 'cat.jpg'
# img = image.load_img(img_path, target_size=(224,224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# print('Predicted:', decode_predictions(preds, top=3)[0])

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=False, download=False, transform=transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = {'plane', 'car', 'bird', 'cat', 'deer',
          'dog', 'frog', 'horse', 'ship', 'truck'}

In [16]:
extra_test_set = np.load("y_test.npy")
extra_testloader = torch.utils.data.DataLoader(extra_test_set, batch_size=4, shuffle=False, num_workers=2)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128*4*4, 256)
        self.drop1 = nn.Dropout2d()
        self.fc2 = nn.Linear(256, 256)
        self.drop2 = nn.Dropout2d()
        self.fc3 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128*4*4)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x
    
model = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)
# optimizer = optim.Adam(self.parameters(), lr=0.001)

In [6]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

[1,  2000] loss: 2.293
[1,  4000] loss: 2.114
[1,  6000] loss: 1.846
[1,  8000] loss: 1.704
[1, 10000] loss: 1.611
[1, 12000] loss: 1.513
[2,  2000] loss: 1.435
[2,  4000] loss: 1.379
[2,  6000] loss: 1.316
[2,  8000] loss: 1.282
[2, 10000] loss: 1.281
[2, 12000] loss: 1.233
[3,  2000] loss: 1.170
[3,  4000] loss: 1.143
[3,  6000] loss: 1.099
[3,  8000] loss: 1.101
[3, 10000] loss: 1.066
[3, 12000] loss: 1.069
[4,  2000] loss: 1.007
[4,  4000] loss: 1.012
[4,  6000] loss: 0.976
[4,  8000] loss: 0.999
[4, 10000] loss: 0.975
[4, 12000] loss: 0.957
[5,  2000] loss: 0.907
[5,  4000] loss: 0.896
[5,  6000] loss: 0.871
[5,  8000] loss: 0.890
[5, 10000] loss: 0.918
[5, 12000] loss: 0.891
Finished Training


In [7]:
# PATH = './cifar_net.path'
# torch.save(model.state_dict(), PATH)

In [11]:
#train error
correct_train = 0
total_train = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

print('Accuracy of the network on the training images: %d %%' %(
    100 * correct_train / total_train))

Accuracy of the network on the training images: 73 %


In [8]:
#test
correct_test = 0
total_test = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(
    100 * correct_test / total_test))

Accuracy of the network on the 10000 test images: 70 %


In [17]:
#extra test
correct_extra_test = 0
total_extra_test = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_extra_test += labels.size(0)
        correct_extra_test += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(
    100 * correct_extra_test / total_extra_test))

Accuracy of the network on the 10000 test images: 70 %
